In [ ]:
app = dash.Dash()
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
server = app.server


#carregamento das bases de covid(federal) e dados estaduais(população, area, localidades, cidades)
df = load_data('brazil_covid19.csv').drop(columns=['region']).rename(columns={'date':'Date','state':'UF','cases':'Cases','deaths':'Deaths'})
brazil = load_data('states.csv').rename({'state': 'State', 'Cities count': 'Cities'}, axis=1)
df = df.merge(brazil, how='left', on='UF')
df = df.drop(["GDP", "Area", "GDP rate", "Demographic Density", "Poverty", "Latitude", "Longitude"], axis=1)

#nem eu sei
c = df[['Date','State','Cases','Deaths']].groupby('Date').sum().reset_index()
c = c[(c['Cases'] >= 1)].melt(id_vars='Date', value_vars=['Cases', 'Deaths'])


#criação das colunas casos por 100mil habitantes, mortes por 100mil habitantes, Subnotificação e Mortalidade    
def subnotif(row):
    d = row['Mortality']
    if d > 0.03: return ((0.97 * row['Deaths']) / 0.034) - row['Cases']
    else: return 0

def engineer_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Mortality'] = df['Deaths']/df['Cases']
    df['Mortality'] = df['Mortality'].fillna(0)
    df['Subnotif'] = df.apply(subnotif, axis=1).astype(int)
    df['Cases per 100K'] = df['Cases']/(df['Population']/10000)
    df['Deaths per 100K'] = df['Deaths']/(df['Population']/10000)
    df = df.fillna(0)
    df = df[['Date', 'UF', 'State','Region', 'Capital', 'Cases', 'Deaths', 'Cases per 100K', 'Deaths per 100K', 'Mortality', 'Subnotif',
             'Population']]
    return df
df = engineer_data(df)

#Agrupando por UF e alimentando a coluna "subnotificação" Considerando a diferença entre a mortalidade global média e a de cada estado
# Com a formula ((0.97*deaths)/0.034) - cases
pd.options.mode.chained_assignment = None
g = df.groupby('UF')
t = g.tail(1).sort_values('Cases', ascending=False).set_index('UF').drop(columns=['Date'])
t['Subnotif'][(t['Subnotif'] < 0)] = 0


#state optoions
state_options = []
for state in df['State'].unique():
    state_options.append({'label':str(state),'value':state})
#layout grafico e dropdown, com opçao "label value"
app.layout = html.Div([
    html.H3('COVID BRASIL'),
    #dcc.Dropdown(id='state-picker',options=state_options,value=df['State'].min()),
    html.Div([
        html.Div([
            dcc.Dropdown(id='state-picker',options=state_options,value=df['State'].min()),
            html.H3('POR ESTADO'),
            dcc.Graph(id='graph')
        ], style={"height" : "40vh", "width" : "50%"}), 
        html.Div([
            html.Pre(id='hover-data', style={'paddingTop':35})
        ], style={'width':'30%'}),
    ]),
    
    html.Div([
        html.H3('POR ESTADO A CADA 100 HABITANTEs'),
        dcc.Graph(id='graph100'),        
    ], style={"height" : "40vh", "width" : "50%"}), 
    html.Div([
        html.Pre(id='hover-data100', style={'paddingTop':35})
    ], style={'width':'30%'})
])
#graph por estado 
@app.callback(Output('graph', 'figure'),
              [Input('state-picker', 'value')])
def update_figure(selected_state):
    #filtra df pelo estado q usuario marcou
    filtered_df = df[df['State'] == selected_state]    
    traces = []
    for region_name in filtered_df['Region'].unique():
        df_by_region = filtered_df[filtered_df['Region'] == region_name]
        traces.append(go.Scatter(
            x=df_by_region['Deaths'],
            y=df_by_region['Cases'],
            text=df_by_region['Date'],
            mode='lines',
            opacity=0.7,
            marker={'size': 15},
            name=region_name
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'type': 'log', 'title': 'MORTES POR ESTADO'},
            yaxis={'title': 'COVID-19: CASOS  POR ESTADO'},
            hovermode='closest'
        )
    }
#por 100k pop
@app.callback(Output('graph100', 'figure'),
              [Input('state-picker', 'value')])
def update_figure(selected_state):
    #filtra df pelo estado q usuario marcou
    filtered_df = df[df['State'] == selected_state]    
    traces = []
    for region_name in filtered_df['Region'].unique():
        df_by_region = filtered_df[filtered_df['Region'] == region_name]
        traces.append(go.Scatter(
            x=df_by_region['Deaths per 100K'],
            y=df_by_region['Cases per 100K'],
            text=df_by_region['Date'],
            mode='lines',
            opacity=0.7,
            marker={'size': 15},
            name=region_name
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'type': 'log', 'title': 'MORTES POR ESTADO A CADA 100K'},
            yaxis={'title': 'COVID-19: CASOS  POR ESTADO A CADA 100K'},
            hovermode='closest'
        )
    }
#retornando texto com o que o usuario estiver olhando
@app.callback(Output('hover-data', 'children'),
              [Input('graph', 'hoverData')])
def callback_image(hoverData):
    deaths=hoverData['points'][0]['y']
    cases=hoverData['points'][0]['x']
    date=hoverData['points'][0]['text']
    hoverData = ['Numero de casos',cases,'Numero de mortes',deaths,'Data',date]    
    return json.dumps(hoverData, indent=2)

@app.callback(Output('hover-data100', 'children'),
              [Input('graph100', 'hoverData')])
#retornando texto com o que o usuario estiver olhando no graph de 100k
def callback_image(hoverData):
    deaths=hoverData['points'][0]['y']
    cases=hoverData['points'][0]['x']
    date=hoverData['points'][0]['text']
    hoverData = ['Numero de casos',cases,'Numero de mortes',deaths,'Data',date]    
    return json.dumps(hoverData, indent=2)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jul/2020 12:00:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:00] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1032, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\anaconda\lib\site-packages\dash\dash

127.0.0.1 - - [06/Jul/2020 12:01:00] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\anaconda\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\anaconda\lib\site-packages\dash\dash.py", line 1032, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\anaconda\lib\site-packages\dash\dash

127.0.0.1 - - [06/Jul/2020 12:01:00] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [06/Jul/2020 12:01:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2020 12:01:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [3]:
pip install virtualenv

Note: you may need to restart the kernel to use updated packages.


In [13]:
my_dash_app>cd

NameError: name 'my_dash_app' is not defined